In [8]:
import sys
import os

import pandas as pd
%matplotlib inline
import urllib.request


import kusto_pandas
from kusto_pandas import Wrap

In [4]:
def dowload_dataset_if_necessary(url, filename):
    if not os.path.exists(filename):
        urllib.request.urlretrieve (url, filename)
url = "https://projects.fivethirtyeight.com/trump-approval-data/approval_polllist.csv"
filename = "approval_polllist.csv"
dowload_dataset_if_necessary(url, filename)
data = pd.read_csv(filename)
data.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,1/20/2021,1/20/2017,1/22/2017,Morning Consult,B/C,1992.0,rv,0.680029,...,37.0,45.686784,38.055805,NaN,NaN,http://static.politico.com/9b/13/82a3baf542ae9...,49249,77261,1/23/2017,11:47:59 20 Jan 2021
1,Donald Trump,All polls,1/20/2021,1/20/2017,1/22/2017,Gallup,B,1500.0,a,0.262323,...,45.0,45.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,11:47:59 20 Jan 2021
2,Donald Trump,All polls,1/20/2021,1/20/2017,1/24/2017,Ipsos,B-,1632.0,a,0.153481,...,45.2,43.451563,43.780389,NaN,T,http://polling.reuters.com/#poll/CP3_2/,49426,77599,3/1/2017,11:47:59 20 Jan 2021
3,Donald Trump,All polls,1/20/2021,1/21/2017,1/23/2017,Gallup,B,1500.0,a,0.242845,...,46.0,45.861441,44.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49262,77274,1/24/2017,11:47:59 20 Jan 2021
4,Donald Trump,All polls,1/20/2021,1/22/2017,1/24/2017,Gallup,B,1500.0,a,0.227380,...,45.0,46.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49236,77248,1/25/2017,11:47:59 20 Jan 2021


In [5]:
w = Wrap(data)

In [10]:
import kusto_pandas.magic

In [11]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic, needs_local_scope)

def _is_table(t):
    return isinstance(t, pd.DataFrame) or isinstance(t, Wrap)

def _get_tables(namespace):
    if namespace is None:
        return dict()
    return dict((k, v) for k, v in namespace.items() if _is_table(v))
    

@register_line_cell_magic
@needs_local_scope
def kusto(line, cell=None, local_ns=None):
    local_tables = _get_tables(local_ns)
    
    cell = cell or ""
    kusto_query = line + " " + cell
    
    wnew = Wrap(pd.DataFrame())
    wnew = wnew.let(**local_tables)
    wnew = wnew.execute(kusto_query)
    return wnew

In [13]:
import kusto_pandas.magic

In [14]:
%%kusto 
w 
| where grade in ('A', 'B')
| summarize count(), avg(disapprove) by grade, pollster
| sort by count_ desc
| take 3

,grade,pollster,count_,avg_disapprove
7,B,YouGov,4582,52.215976
5,B,Gallup,859,55.544820
3,B,American Research Group,127,57.708661


In [15]:
%kusto w | where grade in ('A', 'B') | summarize count(), avg(disapprove) by grade, pollster | sort by count_ desc | take 3 

,grade,pollster,count_,avg_disapprove
7,B,YouGov,4582,52.215976
5,B,Gallup,859,55.544820
3,B,American Research Group,127,57.708661


In [16]:
wnew = %kusto w | where grade in ('A', 'B') | summarize count(), avg(disapprove) by grade, pollster | sort by count_ desc | take 3 

In [17]:
wnew

,grade,pollster,count_,avg_disapprove
7,B,YouGov,4582,52.215976
5,B,Gallup,859,55.544820
3,B,American Research Group,127,57.708661
